In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Data Loading**

In [2]:
import shutil

folder_path = '/content/drive/MyDrive/Articles_Analysis'
shutil.make_archive('/content/Articles_Analysis', 'zip', folder_path)

'/content/Articles_Analysis.zip'

In [3]:
!unzip /content/Articles_Analysis.zip -d /content/Articles_Analysis

Archive:  /content/Articles_Analysis.zip
   creating: /content/Articles_Analysis/MasterDictionary/
   creating: /content/Articles_Analysis/StopWords/
  inflating: /content/Articles_Analysis/Input.xlsx  
  inflating: /content/Articles_Analysis/Text Analysis.docx  
  inflating: /content/Articles_Analysis/Output.xlsx  
  inflating: /content/Articles_Analysis/requirements.txt  
  inflating: /content/Articles_Analysis/Output Data Structure.xlsx  
  inflating: /content/Articles_Analysis/articles.csv  
  inflating: /content/Articles_Analysis/Objective.docx  
  inflating: /content/Articles_Analysis/NLP_Articles_Analysis.ipynb  
  inflating: /content/Articles_Analysis/MasterDictionary/negative-words.txt  
  inflating: /content/Articles_Analysis/MasterDictionary/positive-words.txt  
  inflating: /content/Articles_Analysis/StopWords/StopWords_DatesandNumbers.txt  
  inflating: /content/Articles_Analysis/StopWords/StopWords_Currencies.txt  
  inflating: /content/Articles_Analysis/StopWords/StopWor

## **Data** **Preprocessing**

In [4]:
!pip install -r /content/Artciles_Analysis/requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: '/content/Artciles_Analysis/requirements.txt'


In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df = pd.read_excel('/content/Articles_Analysis/Input.xlsx')

articles = []

for index, row in df.iterrows():
    URL_ID = row[0]
    URL = row[1]

    try:
        response = requests.get(URL)
        soup = BeautifulSoup(response.text, 'html.parser')

        title = soup.find('h1', class_='entry-title')
        title = title.get_text()

        article_text = soup.find('div', class_='td-ss-main-content')
        article_text = article_text.get_text()

        articles.append({
            'URL_ID': URL_ID,
            'URL': URL,
            'Article_Title': title,
            'Article': article_text
        })

    except Exception as e:
        print(f"Error processing URL {URL}: {e}")

articles_df = pd.DataFrame(articles)
articles_df.to_csv('/content/Articles_Analysis/articles.csv', index=False)

<ipython-input-5-c75d520c0af5>:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  URL_ID = row[0]
<ipython-input-5-c75d520c0af5>:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  URL = row[1]


In [6]:
import pandas as pd
import os

df=pd.read_csv('/content/Articles_Analysis/articles.csv')
df.head()

,URL_ID,URL,Article_Title,Article
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,ML and AI-based insurance premium model to pre...,\n\nClient Background\nClient: A leading insur...
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...,Streamlined Integration: Interactive Brokers A...,\n\nClient Background\nClient: A leading finte...
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...,Efficient Data Integration and User-Friendly I...,\n\nClient Background\nClient: A leading tech ...
3,bctech2014,https://insights.blackcoffer.com/effective-man...,Effective Management of Social Media Data Extr...,\n\nClient Background\nClient: A leading tech ...
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...,Streamlined Trading Operations Interface for M...,\n\nClient Background\nClient: A leading finte...


In [7]:
stopword_directory = '/content/Articles_Analysis/StopWords'

def load_stopwords(directory):
    stopwords = list()
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r', encoding='latin-1') as file:
                words = file.read().split()
                stopwords.append(words)
    return stopwords

stop_words = load_stopwords(stopword_directory)
stop_words

[['SMITH',
  '|',
  'Surnames',
  'from',
  '1990',
  'census',
  '>',
  '.002%.',
  'www.census.gov.genealogy/names/dist.all.last',
  'JOHNSON',
  'WILLIAMS',
  'JONES',
  'BROWN',
  'DAVIS',
  'MILLER',
  'WILSON',
  'MOORE',
  'TAYLOR',
  'ANDERSON',
  'THOMAS',
  'JACKSON',
  'WHITE',
  'HARRIS',
  'MARTIN',
  'THOMPSON',
  'GARCIA',
  'MARTINEZ',
  'ROBINSON',
  'CLARK',
  'RODRIGUEZ',
  'LEWIS',
  'LEE',
  'WALKER',
  'HALL',
  'ALLEN',
  'YOUNG',
  'HERNANDEZ',
  'KING',
  'WRIGHT',
  'LOPEZ',
  'HILL',
  'SCOTT',
  'GREEN',
  'ADAMS',
  'BAKER',
  'GONZALEZ',
  'NELSON',
  'CARTER',
  'MITCHELL',
  'PEREZ',
  'ROBERTS',
  'TURNER',
  'PHILLIPS',
  'CAMPBELL',
  'PARKER',
  'EVANS',
  'EDWARDS',
  'COLLINS',
  'STEWART',
  'SANCHEZ',
  'MORRIS',
  'ROGERS',
  'REED',
  'COOK',
  'MORGAN',
  'BELL',
  'MURPHY',
  'BAILEY',
  'RIVERA',
  'COOPER',
  'RICHARDSON',
  'COX',
  'HOWARD',
  'WARD',
  'TORRES',
  'PETERSON',
  'GRAY',
  'RAMIREZ',
  'JAMES',
  'WATSON',
  'BROOKS',
  'K

In [8]:
from nltk.corpus import stopwords

flattened_stopwords = [word for sublist in stop_words for word in sublist]

def remove_stopwords(text):
    words = text.split()  # Split the string into a list of words
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)  # Join the filtered words back into a string

In [9]:
df['Article'] = df['Article'].apply(remove_stopwords)
df.head()

,URL_ID,URL,Article_Title,Article
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,ML and AI-based insurance premium model to pre...,Client Background Client: A leading insurance ...
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...,Streamlined Integration: Interactive Brokers A...,Client Background Client: A leading fintech fi...
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...,Efficient Data Integration and User-Friendly I...,Client Background Client: A leading tech firm ...
3,bctech2014,https://insights.blackcoffer.com/effective-man...,Effective Management of Social Media Data Extr...,Client Background Client: A leading tech firm ...
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...,Streamlined Trading Operations Interface for M...,Client Background Client: A leading fintech fi...


# **Textual Analysis**

In [10]:
positive_dict='/content/Articles_Analysis/MasterDictionary/positive-words.txt'
negative_dict='/content/Articles_Analysis/MasterDictionary/negative-words.txt'

def load_dictionary(file_path):
    with open(file_path, 'r', encoding='latin-1') as file:
        words = file.read().split()
    return words

positive_words = load_dictionary(positive_dict)
negative_words = load_dictionary(negative_dict)

In [11]:
def positive_score(positive_words, text):
    words = text.split()
    positive_count = sum(1 for word in words if word in positive_words)
    return positive_count

def negative_score(negative_words, text):
    words = text.split()
    negative_count = sum(-1 for word in words if word in negative_words)
    return negative_count*-1


df['POSITIVE SCORE'] = df['Article'].apply(lambda x: positive_score(positive_words, x))
df['NEGATIVE SCORE'] = df['Article'].apply(lambda x: negative_score(negative_words, x))
df.head()

,URL_ID,URL,Article_Title,Article,POSITIVE SCORE,NEGATIVE SCORE
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,ML and AI-based insurance premium model to pre...,Client Background Client: A leading insurance ...,83,28
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...,Streamlined Integration: Interactive Brokers A...,Client Background Client: A leading fintech fi...,15,2
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...,Efficient Data Integration and User-Friendly I...,Client Background Client: A leading tech firm ...,18,3
3,bctech2014,https://insights.blackcoffer.com/effective-man...,Effective Management of Social Media Data Extr...,Client Background Client: A leading tech firm ...,9,2
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...,Streamlined Trading Operations Interface for M...,Client Background Client: A leading fintech fi...,9,1


In [12]:
def polarity_score(positive_score, negative_score):
    return (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)

df['POLARITY SCORE'] = df.apply(lambda row: polarity_score(row['POSITIVE SCORE'], row['NEGATIVE SCORE']), axis=1)
df.head()

,URL_ID,URL,Article_Title,Article,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,ML and AI-based insurance premium model to pre...,Client Background Client: A leading insurance ...,83,28,0.495495
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...,Streamlined Integration: Interactive Brokers A...,Client Background Client: A leading fintech fi...,15,2,0.764706
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...,Efficient Data Integration and User-Friendly I...,Client Background Client: A leading tech firm ...,18,3,0.714286
3,bctech2014,https://insights.blackcoffer.com/effective-man...,Effective Management of Social Media Data Extr...,Client Background Client: A leading tech firm ...,9,2,0.636364
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...,Streamlined Trading Operations Interface for M...,Client Background Client: A leading fintech fi...,9,1,0.800000


In [13]:
def subjectivity_score(positive_score, negative_score, num_words):
    return (positive_score + negative_score) / ((num_words) + 0.000001)

df['SUBJECTIVITY SCORE']=df.apply(lambda row: subjectivity_score(row['POSITIVE SCORE'], row['NEGATIVE SCORE'], len(row['Article'].split())), axis=1)
df.head()

,URL_ID,URL,Article_Title,Article,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,ML and AI-based insurance premium model to pre...,Client Background Client: A leading insurance ...,83,28,0.495495,0.042692
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...,Streamlined Integration: Interactive Brokers A...,Client Background Client: A leading fintech fi...,15,2,0.764706,0.032258
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...,Efficient Data Integration and User-Friendly I...,Client Background Client: A leading tech firm ...,18,3,0.714286,0.033493
3,bctech2014,https://insights.blackcoffer.com/effective-man...,Effective Management of Social Media Data Extr...,Client Background Client: A leading tech firm ...,9,2,0.636364,0.021654
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...,Streamlined Trading Operations Interface for M...,Client Background Client: A leading fintech fi...,9,1,0.800000,0.016155


In [14]:
def analyis_of_readability(text):
    words = text.split()
    num_words = len(words)
    num_sentences = len(text.split('.'))
    avg_sentence_length = num_words / num_sentences
    percentage_complex_words = sum(1 for word in words if len(word) > 6) / num_words
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)
    # Return a Series to match the expected output for assigning to multiple columns
    return pd.Series({
        'AVG SENTENCE LENGTH': avg_sentence_length,
        'PERCENTAGE OF COMPLEX WORDS': percentage_complex_words,
        'FOG INDEX': fog_index
    })

# Remove the axis argument as it is not needed here
df[['AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX']] = df['Article'].apply(analyis_of_readability)
df.head()

,URL_ID,URL,Article_Title,Article,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,ML and AI-based insurance premium model to pre...,Client Background Client: A leading insurance ...,83,28,0.495495,0.042692,12.621359,0.533077,5.261774
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...,Streamlined Integration: Interactive Brokers A...,Client Background Client: A leading fintech fi...,15,2,0.764706,0.032258,9.086207,0.529412,3.846247
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...,Efficient Data Integration and User-Friendly I...,Client Background Client: A leading tech firm ...,18,3,0.714286,0.033493,13.933333,0.476874,5.764083
3,bctech2014,https://insights.blackcoffer.com/effective-man...,Effective Management of Social Media Data Extr...,Client Background Client: A leading tech firm ...,9,2,0.636364,0.021654,8.758621,0.521654,3.712110
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...,Streamlined Trading Operations Interface for M...,Client Background Client: A leading fintech fi...,9,1,0.800000,0.016155,17.194444,0.442649,7.054838


In [15]:
def average_words_per_sentence(text):
    words = text.split()
    num_words = len(words)
    num_sentences = len(text.split('.'))
    return num_words / num_sentences

df['AVG NUMBER OF WORDS PER SENTENCE'] = df['Article'].apply(average_words_per_sentence)
df.head()


,URL_ID,URL,Article_Title,Article,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,ML and AI-based insurance premium model to pre...,Client Background Client: A leading insurance ...,83,28,0.495495,0.042692,12.621359,0.533077,5.261774,12.621359
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...,Streamlined Integration: Interactive Brokers A...,Client Background Client: A leading fintech fi...,15,2,0.764706,0.032258,9.086207,0.529412,3.846247,9.086207
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...,Efficient Data Integration and User-Friendly I...,Client Background Client: A leading tech firm ...,18,3,0.714286,0.033493,13.933333,0.476874,5.764083,13.933333
3,bctech2014,https://insights.blackcoffer.com/effective-man...,Effective Management of Social Media Data Extr...,Client Background Client: A leading tech firm ...,9,2,0.636364,0.021654,8.758621,0.521654,3.712110,8.758621
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...,Streamlined Trading Operations Interface for M...,Client Background Client: A leading fintech fi...,9,1,0.800000,0.016155,17.194444,0.442649,7.054838,17.194444


In [16]:
import nltk
nltk.download('punkt')
from nltk.corpus import cmudict
nltk.download('cmudict')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


True

In [17]:
syllable_dict = cmudict.dict()

def count_syllables(word):
    if word.lower() not in syllable_dict:    # search for lower case version of the word in dictionary
        return 0
    return [len(list(y for y in x if y[-1].isdigit())) for x in syllable_dict[word.lower()]][0]
                                               # return number of syllable

def is_complex(word):
    syllable_count = count_syllables(word)
    return syllable_count > 2

def count_complex_words(text):
    words = nltk.word_tokenize(text)
    num_complex_words = sum(is_complex(word) for word in words)
    return num_complex_words

df['COMPLEX WORD COUNT'] = df['Article'].apply(count_complex_words)
df.head()

,URL_ID,URL,Article_Title,Article,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,ML and AI-based insurance premium model to pre...,Client Background Client: A leading insurance ...,83,28,0.495495,0.042692,12.621359,0.533077,5.261774,12.621359,872
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...,Streamlined Integration: Interactive Brokers A...,Client Background Client: A leading fintech fi...,15,2,0.764706,0.032258,9.086207,0.529412,3.846247,9.086207,162
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...,Efficient Data Integration and User-Friendly I...,Client Background Client: A leading tech firm ...,18,3,0.714286,0.033493,13.933333,0.476874,5.764083,13.933333,152
3,bctech2014,https://insights.blackcoffer.com/effective-man...,Effective Management of Social Media Data Extr...,Client Background Client: A leading tech firm ...,9,2,0.636364,0.021654,8.758621,0.521654,3.712110,8.758621,149
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...,Streamlined Trading Operations Interface for M...,Client Background Client: A leading fintech fi...,9,1,0.800000,0.016155,17.194444,0.442649,7.054838,17.194444,154


In [18]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
import string
from nltk.corpus import stopwords
stop_words_=stopwords.words("english")

def remove__stopwords(text):
    words = text.split()  # Split the string into a list of words
    filtered_words = [word for word in words if word not in stop_words_]
    return ' '.join(filtered_words)

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def word_count(text):
    text = remove_punctuation(text)
    text= remove__stopwords(text)
    words = text.split()
    return len(words)

df['WORD COUNT'] = df['Article'].apply(word_count)
df.head()

,URL_ID,URL,Article_Title,Article,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,ML and AI-based insurance premium model to pre...,Client Background Client: A leading insurance ...,83,28,0.495495,0.042692,12.621359,0.533077,5.261774,12.621359,872,1950
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...,Streamlined Integration: Interactive Brokers A...,Client Background Client: A leading fintech fi...,15,2,0.764706,0.032258,9.086207,0.529412,3.846247,9.086207,162,425
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...,Efficient Data Integration and User-Friendly I...,Client Background Client: A leading tech firm ...,18,3,0.714286,0.033493,13.933333,0.476874,5.764083,13.933333,152,504
3,bctech2014,https://insights.blackcoffer.com/effective-man...,Effective Management of Social Media Data Extr...,Client Background Client: A leading tech firm ...,9,2,0.636364,0.021654,8.758621,0.521654,3.712110,8.758621,149,421
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...,Streamlined Trading Operations Interface for M...,Client Background Client: A leading fintech fi...,9,1,0.800000,0.016155,17.194444,0.442649,7.054838,17.194444,154,464


In [20]:
def count_syllables(word):
    word = word.lower()
    num_vowels = len([char for char in word if char in 'aeiou'])
    if word.endswith('es') or word.endswith('ed'):
        num_vowels -= 1           # do not include 'es' and 'ed' of found
    return num_vowels

def count_syllables_per_word(text):
    words = nltk.word_tokenize(text)
    syllable_counts = [count_syllables(word) for word in words]
    return syllable_counts

df['SYLLABLE PER WORD'] = df['Article'].apply(count_syllables_per_word)
df.head()

,URL_ID,URL,Article_Title,Article,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,ML and AI-based insurance premium model to pre...,Client Background Client: A leading insurance ...,83,28,0.495495,0.042692,12.621359,0.533077,5.261774,12.621359,872,1950,"[2, 3, 2, 0, 1, 3, 4, 1, 3, 2, 1, 0, 1, 2, 0, ..."
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...,Streamlined Integration: Interactive Brokers A...,Client Background Client: A leading fintech fi...,15,2,0.764706,0.032258,9.086207,0.529412,3.846247,9.086207,162,425,"[2, 3, 2, 0, 1, 3, 2, 1, 1, 1, 2, 2, 1, 0, 3, ..."
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...,Efficient Data Integration and User-Friendly I...,Client Background Client: A leading tech firm ...,18,3,0.714286,0.033493,13.933333,0.476874,5.764083,13.933333,152,504,"[2, 3, 2, 0, 1, 3, 1, 1, 1, 1, 2, 2, 1, 0, 1, ..."
3,bctech2014,https://insights.blackcoffer.com/effective-man...,Effective Management of Social Media Data Extr...,Client Background Client: A leading tech firm ...,9,2,0.636364,0.021654,8.758621,0.521654,3.712110,8.758621,149,421,"[2, 3, 2, 0, 1, 3, 1, 1, 1, 1, 2, 2, 1, 0, 1, ..."
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...,Streamlined Trading Operations Interface for M...,Client Background Client: A leading fintech fi...,9,1,0.800000,0.016155,17.194444,0.442649,7.054838,17.194444,154,464,"[2, 3, 2, 0, 1, 3, 2, 1, 1, 1, 2, 2, 1, 0, 3, ..."


In [21]:
import re
def count_personal_pronouns(text):
    pattern = r"\b(I|we|my|ours|us)\b"   # pattern to check if those words exists
    pattern = r"(?<!\bUS\b)" + pattern   # pattern should not include US instead of us
    matches = re.findall(pattern, text, flags=re.IGNORECASE)
    count = len(matches)
    return count
df['PERSONAL PRONOUNS'] = df['Article'].apply(count_personal_pronouns)
df.head()

,URL_ID,URL,Article_Title,Article,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,ML and AI-based insurance premium model to pre...,Client Background Client: A leading insurance ...,83,28,0.495495,0.042692,12.621359,0.533077,5.261774,12.621359,872,1950,"[2, 3, 2, 0, 1, 3, 4, 1, 3, 2, 1, 0, 1, 2, 0, ...",2
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...,Streamlined Integration: Interactive Brokers A...,Client Background Client: A leading fintech fi...,15,2,0.764706,0.032258,9.086207,0.529412,3.846247,9.086207,162,425,"[2, 3, 2, 0, 1, 3, 2, 1, 1, 1, 2, 2, 1, 0, 3, ...",1
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...,Efficient Data Integration and User-Friendly I...,Client Background Client: A leading tech firm ...,18,3,0.714286,0.033493,13.933333,0.476874,5.764083,13.933333,152,504,"[2, 3, 2, 0, 1, 3, 1, 1, 1, 1, 2, 2, 1, 0, 1, ...",1
3,bctech2014,https://insights.blackcoffer.com/effective-man...,Effective Management of Social Media Data Extr...,Client Background Client: A leading tech firm ...,9,2,0.636364,0.021654,8.758621,0.521654,3.712110,8.758621,149,421,"[2, 3, 2, 0, 1, 3, 1, 1, 1, 1, 2, 2, 1, 0, 1, ...",1
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...,Streamlined Trading Operations Interface for M...,Client Background Client: A leading fintech fi...,9,1,0.800000,0.016155,17.194444,0.442649,7.054838,17.194444,154,464,"[2, 3, 2, 0, 1, 3, 2, 1, 1, 1, 2, 2, 1, 0, 3, ...",1


In [22]:
def calculate_avg_word_length(text):
    words = text.split()
    total_characters = sum(len(word) for word in words)  # sum of the charachter
    num_words = len(words)                               # total of words
    avg_word_length = total_characters / num_words
    return avg_word_length
df['AVG WORD LENGTH'] = df['Article'].apply(calculate_avg_word_length)
df.head()

,URL_ID,URL,Article_Title,Article,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,ML and AI-based insurance premium model to pre...,Client Background Client: A leading insurance ...,83,28,0.495495,0.042692,12.621359,0.533077,5.261774,12.621359,872,1950,"[2, 3, 2, 0, 1, 3, 4, 1, 3, 2, 1, 0, 1, 2, 0, ...",2,6.640000
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...,Streamlined Integration: Interactive Brokers A...,Client Background Client: A leading fintech fi...,15,2,0.764706,0.032258,9.086207,0.529412,3.846247,9.086207,162,425,"[2, 3, 2, 0, 1, 3, 2, 1, 1, 1, 2, 2, 1, 0, 3, ...",1,7.149905
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...,Efficient Data Integration and User-Friendly I...,Client Background Client: A leading tech firm ...,18,3,0.714286,0.033493,13.933333,0.476874,5.764083,13.933333,152,504,"[2, 3, 2, 0, 1, 3, 1, 1, 1, 1, 2, 2, 1, 0, 1, ...",1,6.752791
3,bctech2014,https://insights.blackcoffer.com/effective-man...,Effective Management of Social Media Data Extr...,Client Background Client: A leading tech firm ...,9,2,0.636364,0.021654,8.758621,0.521654,3.712110,8.758621,149,421,"[2, 3, 2, 0, 1, 3, 1, 1, 1, 1, 2, 2, 1, 0, 1, ...",1,6.925197
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...,Streamlined Trading Operations Interface for M...,Client Background Client: A leading fintech fi...,9,1,0.800000,0.016155,17.194444,0.442649,7.054838,17.194444,154,464,"[2, 3, 2, 0, 1, 3, 2, 1, 1, 1, 2, 2, 1, 0, 3, ...",1,6.326333


# **Saving the Ouput File**

In [23]:
df1=df.copy()
df1.drop(['Article', 'Article_Title'], axis=1, inplace=True)
df1.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,83,28,0.495495,0.042692,12.621359,0.533077,5.261774,12.621359,872,1950,"[2, 3, 2, 0, 1, 3, 4, 1, 3, 2, 1, 0, 1, 2, 0, ...",2,6.640000
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...,15,2,0.764706,0.032258,9.086207,0.529412,3.846247,9.086207,162,425,"[2, 3, 2, 0, 1, 3, 2, 1, 1, 1, 2, 2, 1, 0, 3, ...",1,7.149905
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...,18,3,0.714286,0.033493,13.933333,0.476874,5.764083,13.933333,152,504,"[2, 3, 2, 0, 1, 3, 1, 1, 1, 1, 2, 2, 1, 0, 1, ...",1,6.752791
3,bctech2014,https://insights.blackcoffer.com/effective-man...,9,2,0.636364,0.021654,8.758621,0.521654,3.712110,8.758621,149,421,"[2, 3, 2, 0, 1, 3, 1, 1, 1, 1, 2, 2, 1, 0, 1, ...",1,6.925197
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...,9,1,0.800000,0.016155,17.194444,0.442649,7.054838,17.194444,154,464,"[2, 3, 2, 0, 1, 3, 2, 1, 1, 1, 2, 2, 1, 0, 3, ...",1,6.326333


In [24]:
import openpyxl

df1.to_excel('/content/Articles_Analysis/Output.xlsx', index=False)

In [25]:
import shutil

base_name = '/content/Articles_Analysis'

shutil.make_archive(base_name, 'zip', '/content/Articles_Analysis')

print(f"Archive created at: {base_name}.zip")

Archive created at: /content/Articles_Analysis.zip
